# Lithological Classification of Well Logs Using Machine Learning

## 1. Project Overview
This notebook implements a machine learning workflow to classify lithofacies from well log measurements using the FORCE 2020 dataset. The methodology employs **Gradient Boosting (XGBoost)**, enhanced by domain-specific feature engineering (petrophysical ratios) and vectorized data augmentation (spatial windowing).

### Objectives
- Load and preprocess well log data.
- Engineer features based on rock physics principles.
- Train an XGBoost classifier using GPU acceleration.
- Evaluate model performance on a blind test set.

## 2. Library Import and Environment Setup
The following cell initializes the Python environment by importing necessary libraries for data manipulation, visualization, and machine learning.

**Technical Configuration:**
* **Garbage Collection (`gc`):** Imported to explicitly manage memory during the iterative cross-validation process.
* **Warning Suppression:** Enabled to maintain clean output logs during training.
* **Visualization:** Seaborn is configured with the 'ticks' style for academic-grade plotting.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import gc
import warnings
import time

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sns.set_style("ticks")

## 3. Data Acquisition
The dataset is loaded directly from the source repository. To ensure data integrity for validation, the test features are merged with their corresponding target labels (`test_target.csv`) using the unique well identifier and depth (`DEPTH_MD`).

**Note:** The training set is large (~1.17M rows), making efficient memory management crucial.

In [ ]:
# ==================================================================================
# 1. Data Loading (Raw Data)
# ==================================================================================
print("⏳ Loading datasets directly from source...")
start_time = time.time()

# Load raw data directly to bypass previous bad imputation
train = pd.read_csv('https://media.githubusercontent.com/media/Husayn01/Academic-Research/refs/heads/main/Data/train.csv', sep=';')
test = pd.read_csv('https://media.githubusercontent.com/media/Husayn01/Academic-Research/refs/heads/main/Data/test_features.csv', sep=';')
test_target = pd.read_csv('https://media.githubusercontent.com/media/Husayn01/Academic-Research/refs/heads/main/Data/test_target.csv', sep=';')

# Merge test target
test = test.merge(test_target, on=['WELL', 'DEPTH_MD'], how='left')

print(f"✅ Data Loaded in {time.time() - start_time:.2f}s")
print(f"   Train shape: {train.shape}")
print(f"   Test shape: {test.shape}")

target_col = 'FORCE_2020_LITHOFACIES_LITHOLOGY'

## 4. Strategic Feature Engineering
Raw well logs often contain non-linear relationships or noise that hinder model performance. This section applies petrophysical transformations to extract meaningful geological signals.

### Transformations Applied:
1.  **Dimensionality Reduction:** Sparse columns (e.g., `SGR`, `RXO`, `ROPA`) are dropped to prevent the model from learning noise from missing values.
2.  **Logarithmic Scaling:** Applied to resistivity logs (`RDEP`, `RMED`) because earth conductivity varies over multiple orders of magnitude.
3.  **Geological Ratios:**
    * **`GR_RHOB`:** Helps distinguish between shale volume (high GR) and compaction effects (high density).
    * **Neutron-Density Separation (`NPHI - RHOB`):** A classic "balloon effect" indicator used to identify gas zones and differentiate sandstone from limestone.
    * **Photoelectric Effect (`PEF^2`):** Squared to emphasize sensitivity to mineralogy (atomic number).

In [ ]:
# ==================================================================================
# 2. Strategic Feature Engineering
# ==================================================================================
def engineer_features(df):
    df = df.copy()

    # A. Drop sparse/noisy columns (Winning strategy from competition)
    # SGR, ROPA, RXO etc. had too many missing values and introduced noise
    drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT', 'DCAL', 'RMIC', 'FORCE_2020_LITHOFACIES_CONFIDENCE']
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

    # B. Log transformations for resistivity (Standard Petrophysics)
    # Resistivity spans logarithmic scales (0.2 to 2000+ ohm.m)
    for col in ['RDEP', 'RMED', 'RSHA']:
        if col in df.columns:
            df[f'{col}_LOG'] = np.log1p(df[col])

    # C. Domain Ratios & Differences
    # GR/RHOB: Shale indicator vs Compaction
    if 'GR' in df.columns and 'RHOB' in df.columns:
        df['GR_RHOB'] = df['GR'] / (df['RHOB'] + 0.001)

    # Neutron-Density Separation (Gas Effect/Lithology)
    if 'NPHI' in df.columns and 'RHOB' in df.columns:
        # Approx matrix density 2.65 for Sandstone
        df['NPHI_RHOB_DIFF'] = df['NPHI'] - (2.65 - df['RHOB'])

    # Photoelectric Effect squared (Mineralogy)
    if 'PEF' in df.columns:
        df['PEF_SQ'] = df['PEF'] ** 2

    # D. Fill MISSING categorical values string before encoding
    # We do NOT impute numerical values. XGBoost handles NaNs natively.
    if 'GROUP' in df.columns:
        df['GROUP'] = df['GROUP'].fillna('Unknown')
    if 'FORMATION' in df.columns:
        df['FORMATION'] = df['FORMATION'].fillna('Unknown')

    return df

print("🛠️ Engineering base features...")
train_eng = engineer_features(train)
test_eng = engineer_features(test)

## 5. Vectorized Data Augmentation (Spatial Windowing)
Geological data is spatially correlated; the lithology at depth $d$ is highly influenced by the layers at $d-1$ and $d+1$. We implement a **windowing strategy** to incorporate vertical context.

**Implementation Detail:**
Instead of slow iterative loops, we use vectorized `shift` operations grouped by `WELL` ID. This creates lag (previous) and lead (next) features, as well as gradient features (instantaneous change) to detect bed boundaries.

In [ ]:
# ==================================================================================
# 3. Vectorized Augmentation (Windowing)
# ==================================================================================
def vectorized_windowing(df, cols, window_size=1):
    """Creates lag/lead features using vectorized shifts (faster than rolling)."""
    df_out = df.copy()
    # Sort to ensure depth continuity
    df_out = df_out.sort_values(['WELL', 'DEPTH_MD'])

    for col in cols:
        if col not in df_out.columns:
            continue
        # Create lags (above) and leads (below)
        for i in range(1, window_size + 1):
            df_out[f'{col}_prev_{i}'] = df_out.groupby('WELL')[col].shift(i)
            df_out[f'{col}_next_{i}'] = df_out.groupby('WELL')[col].shift(-i)
        # Gradients (Instantaneous change)
        df_out[f'{col}_grad'] = df_out[col] - df_out.groupby('WELL')[col].shift(1)

    return df_out

print("🪟 Applying window augmentation...")
aug_cols = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP_LOG']
train_final = vectorized_windowing(train_eng, aug_cols, window_size=1)
test_final = vectorized_windowing(test_eng, aug_cols, window_size=1)

print(f"   Final Train shape: {train_final.shape}")

## 6. Preprocessing and Encoding
Before training, categorical variables are encoded into integer formats required by XGBoost. Additionally, column names are sanitized to remove special characters (like brackets `[]` common in LAS files) which can cause parsing errors in the model.

**Steps:**
1.  **Label Encoding:** `GROUP` and `FORMATION` columns are fitted on the combined dataset to handle potential unseen labels in the test set (a strategy often used in competitions, though technically a form of minor leakage; see review below).
2.  **Target Encoding:** The lithology labels are transformed into a 0-N range.
3.  **Feature Split:** Dropping non-predictive metadata (`DEPTH_MD`, `X_LOC`, etc.).

In [ ]:
# ==================================================================================
# 4. Encoding & Final Prep
# ==================================================================================
# Encode Categoricals
cat_cols = ['GROUP', 'FORMATION', 'WELL']
for col in cat_cols:
    if col in train_final.columns:
        le = LabelEncoder()
        # Fit on both train and test to catch all categories
        unique_vals = pd.concat([train_final[col], test_final[col]]).astype(str).unique()
        le.fit(unique_vals)
        train_final[col] = le.transform(train_final[col].astype(str))
        test_final[col] = le.transform(test_final[col].astype(str))

# Encode Target
le_target = LabelEncoder()
train_final[target_col] = le_target.fit_transform(train_final[target_col])
test_final[target_col] = le_target.transform(test_final[target_col])

# Define X and y
drop_for_train = ['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', target_col]
X = train_final.drop(columns=drop_for_train)
y = train_final[target_col]
X_test_data = test_final.drop(columns=drop_for_train)
y_test_data = test_final[target_col]

# Clean column names for XGBoost
X.columns = [c.replace('[', '_').replace(']', '_') for c in X.columns]
X_test_data.columns = [c.replace('[', '_').replace(']', '_') for c in X_test_data.columns]

print("✨ Features ready.")

## 7. Model Training: XGBoost with Stratified K-Fold
The model is trained using **Stratified 10-Fold Cross-Validation**. Stratification is essential here due to the severe class imbalance in geological datasets (e.g., massive Shale intervals vs. thin Coal or Basement layers).

**Model Parameters:**
* `tree_method='hist'`, `device='cuda'`: Enables **GPU acceleration** for significantly faster training on large datasets.
* `objective='multi:softprob'`: Used for multi-class classification to output probabilities.

In [ ]:
# ==================================================================================
# 5. Training with GPU (XGBoost 2.0+ Compatible)
# ==================================================================================
# Model Parameters (Optimized for GPU)
xgb_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.5,
    'random_state': 42,
    'objective': 'multi:softprob',
    'early_stopping_rounds': 30,
    'n_jobs': -1,
    'reg_lambda': 1.5,
    'reg_alpha': 0.5,
    # --- CRITICAL GPU SETTINGS FOR XGBOOST 2.0+ ---
    'tree_method': 'hist',
    'device': 'cuda'
}

folds = 10
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

test_preds_accum = np.zeros((len(X_test_data), 12))
scores = []

print(f"🚀 Starting {folds}-Fold Cross-Validation on T4 GPU...")
cv_start = time.time()

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    fold_start = time.time()
    X_fold_train, y_fold_train = X.iloc[train_idx], y.iloc[train_idx]
    X_fold_val, y_fold_val = X.iloc[val_idx], y.iloc[val_idx]

    model = XGBClassifier(**xgb_params)

    model.fit(
        X_fold_train, y_fold_train,
        eval_set=[(X_fold_val, y_fold_val)],
        verbose=False
    )

    # Predict
    val_probs = model.predict_proba(X_fold_val)
    test_probs = model.predict_proba(X_test_data)
    test_preds_accum += test_probs / folds

    # Score
    val_pred_labels = np.argmax(val_probs, axis=1)
    acc = accuracy_score(y_fold_val, val_pred_labels)
    scores.append(acc)

    print(f"   Fold {fold+1}/{folds} | Accuracy: {acc:.4f} | Time: {time.time() - fold_start:.1f}s")

    del X_fold_train, X_fold_val, model
    gc.collect()

print("="*40)
print(f"✅ Average CV Accuracy: {np.mean(scores):.4f}")
print(f"⏱️ Total CV Time: {(time.time() - cv_start)/60:.2f} minutes")
print("="*40)

# ==================================================================================
# 6. Final Evaluation
# ==================================================================================
# Final Predictions
final_test_labels = np.argmax(test_preds_accum, axis=1)

# Metrics
test_acc = accuracy_score(y_test_data, final_test_labels)
test_f1 = f1_score(y_test_data, final_test_labels, average='weighted')

print(f"\n🏆 Final Test Accuracy: {test_acc:.4f}")
print(f"🏆 Final Test F1 (Weighted): {test_f1:.4f}")

## 8. Performance Evaluation and Visualization
We convert the numeric predictions back to geological lithology names and generate a classification report. A confusion matrix is plotted to analyze misclassifications, particularly between geologically similar classes (e.g., Sandstone vs. Sandstone/Shale).

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Lithology mapping for readable labels
lithology_keys = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale',
                  80000: 'Marl', 74000: 'Dolomite', 70000: 'Limestone',
                  70032: 'Chalk', 88000: 'Halite', 86000: 'Anhydrite',
                  99000: 'Tuff', 90000: 'Coal', 93000: 'Basement'}

# Create friendly labels mapping original codes to names
target_names_mapped = [lithology_keys.get(code, str(code)) for code in le_target.classes_]

print("\nClassification Report:")
# FIX: Provide explicit labels to handle missing classes in test set
all_labels = np.arange(len(le_target.classes_))
print(classification_report(
    y_test_data,
    final_test_labels,
    labels=all_labels,
    target_names=target_names_mapped,
    zero_division=0
))

# Optional: Confusion Matrix Plot
try:
    # FIX: Pass labels=all_labels to ensure the matrix shape matches the target names,
    # even if some classes are missing in y_test_data.
    cm = confusion_matrix(y_test_data, final_test_labels, labels=all_labels)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names_mapped,
                yticklabels=target_names_mapped)
    plt.title(f'Confusion Matrix (Acc:)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
except Exception as e:
    print(f"Could not plot confusion matrix: {e}")

## 9. Technical Review & Conclusion

### Model Behavior
* **Generalization:** The discrepancy between CV accuracy (~93%) and Test accuracy (~78%) suggests a **distribution shift** between the training wells and the blind test wells. This is common in subsurface data where geological trends vary spatially (e.g., North vs. South North Sea).
* **Class Imbalance:** High-frequency classes (Shale, Sandstone) perform well (F1 > 0.8). Rare classes (Dolomite, Coal, Basement) suffer from low recall (F1 ~ 0.0).

### Identified Issues & Recommendations
* **Geographic Overfitting:** Including `X_LOC` and `Y_LOC` often causes trees to overfit to specific coordinates rather than learning rock physics. **Recommendation:** Experiment with removing coordinates or replacing them with basin-relative trends.
* **Label Leakage:** Fitting the `LabelEncoder` on the combined Train+Test set (`pd.concat`) allows the model to anticipate categories in the test set. While harmless for label mapping, ensuring the model can handle "Unknown" categories during inference is critical for production.
* **Mixed Lithologies:** The class "Sandstone/Shale" has low recall (often confused with pure Sandstone or Shale). This represents a physical ambiguity in the logs that standard trees struggle to resolve without higher-resolution data.